In [1]:
import torch                              
import torch.nn as nn                      
import torch.optim as optim                
from torch.utils.data import DataLoader    
import torchvision                       
import torchvision.transforms as transforms 
import numpy as np                         
import cv2                                
from skimage.metrics import structural_similarity as ssim  
import matplotlib.pyplot as plt           
import time

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
def calculate_psnr(denoised, ground_truth):
    mse = np.mean((denoised - ground_truth) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 1.0 
    psnr = 20 * np.log10(PIXEL_MAX / np.sqrt(mse))
    return psnr
from skimage.metrics import structural_similarity as ssim


def calculate_ssim(denoised, ground_truth):
    return ssim(ground_truth, denoised, data_range=ground_truth.max() - ground_truth.min(), win_size=7, channel_axis=-1)


In [4]:
import torch
import torch.nn as nn

class DnCNN(nn.Module):
    """
    DnCNN (Denoising Convolutional Neural Network) is a deep learning model used for image denoising.
    It learns to predict the noise residual and subtract it from the input image to recover the clean image.
    """
    def __init__(self, channels=3, num_of_layers=17, features=64):
        super(DnCNN, self).__init__()
        layers = [] 

        layers.append(nn.Conv2d(in_channels=channels, out_channels=features, kernel_size=3, padding=1, bias=True))
        layers.append(nn.ReLU(inplace=True)) 

        for _ in range(num_of_layers - 2):
            layers.append(nn.Conv2d(in_channels=features, out_channels=features, kernel_size=3, padding=1, bias=False))  
            layers.append(nn.BatchNorm2d(features))  
            layers.append(nn.ReLU(inplace=True))  

        layers.append(nn.Conv2d(in_channels=features, out_channels=channels, kernel_size=3, padding=1, bias=False)) 
        self.dncnn = nn.Sequential(*layers)

    def forward(self, x):
        """
        The forward pass takes an input tensor (x), applies the DnCNN network, and subtracts the predicted noise
        from the input to output the denoised image.
        """
        noise = self.dncnn(x) 
        return x - noise  


In [5]:
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 128
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# test_loader  = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
model = DnCNN(channels=3).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 20
noise_std = 0.1  

In [7]:
print("Starting Training...")
model.train()  
best_loss = float('inf')
patience = 2  
patience_counter = 0
best_model_state = None


for epoch in range(num_epochs):
    epoch_loss = 0
    start_time = time.time()

    for data, _ in train_loader:
        data = data.to(device)  
        noise = torch.randn_like(data) * noise_std
        noisy_data = data + noise
        output = model(noisy_data)
        loss = criterion(output, data)
        epoch_loss += loss.item() * data.size(0)

        optimizer.zero_grad()  
        loss.backward()     
        optimizer.step()       

    epoch_loss /= len(train_dataset)
    elapsed = time.time() - start_time
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.6f}, Time: {elapsed:.2f} sec")

    # Early stopping check
    if epoch_loss < best_loss - 1e-6:  
        best_loss = epoch_loss
        patience_counter = 0
        best_model_state = model.state_dict() 
    else:
        patience_counter += 1
        print(f"No improvement. Patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print("Loaded best model with lowest validation loss.")


Starting Training...
Epoch [1/20], Loss: 0.010775, Time: 73.54 sec
Epoch [2/20], Loss: 0.004080, Time: 70.58 sec
Epoch [3/20], Loss: 0.002379, Time: 70.63 sec
Epoch [4/20], Loss: 0.002127, Time: 68.28 sec
Epoch [5/20], Loss: 0.001918, Time: 67.57 sec
Epoch [6/20], Loss: 0.001874, Time: 67.63 sec
Epoch [7/20], Loss: 0.001813, Time: 67.40 sec
Epoch [8/20], Loss: 0.001724, Time: 67.56 sec
Epoch [9/20], Loss: 0.001658, Time: 67.47 sec
Epoch [10/20], Loss: 0.001636, Time: 67.52 sec
Epoch [11/20], Loss: 0.001787, Time: 69.58 sec
No improvement. Patience: 1/2
Epoch [12/20], Loss: 0.001620, Time: 69.92 sec
Epoch [13/20], Loss: 0.001575, Time: 67.42 sec
Epoch [14/20], Loss: 0.001548, Time: 67.45 sec
Epoch [15/20], Loss: 0.001920, Time: 67.45 sec
No improvement. Patience: 1/2
Epoch [16/20], Loss: 0.010073, Time: 69.39 sec
No improvement. Patience: 2/2
Early stopping triggered.
Loaded best model with lowest validation loss.


In [9]:

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

class ImageFolderNoClass(Dataset):
    def __init__(self, folder_path, transform=None):
        self.file_paths = [os.path.join(folder_path, f) 
                           for f in os.listdir(folder_path) 
                           if f.lower().endswith(('png', 'jpg', 'jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img = Image.open(self.file_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, 0

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32, 32))
])

train_dataset = ImageFolderNoClass('./BSD500/train', transform=transform)
val_dataset   = ImageFolderNoClass('./BSD500/val', transform=transform)
test_dataset  = ImageFolderNoClass('./BSD500/test', transform=transform)

# batch_size = 32
# train_loader = DataLoader(train_dataset+val_dataset, batch_size=batch_size, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(train_dataset+val_dataset+test_dataset, batch_size=batch_size, shuffle=False)


model.eval()
psnr_list = []
ssim_list = []


with torch.no_grad():
    for data, _ in test_loader:
        data = data.to(device)
        noise = torch.randn_like(data) * noise_std
        noisy_data = data + noise
        output = model(noisy_data)
        
        # Move tensors to CPU and convert to numpy arrays, clipping values into [0,1]
        output_np = output.cpu().numpy().transpose(0, 2, 3, 1)   # (N, H, W, C)
        clean_np  = data.cpu().numpy().transpose(0, 2, 3, 1)
        noisy_np  = noisy_data.cpu().numpy().transpose(0, 2, 3, 1)
        
        # Calculate metrics image by image
        for denoised, clean in zip(output_np, clean_np):
            denoised = np.clip(denoised, 0., 1.)
            clean = np.clip(clean, 0., 1.)
            psnr_val = calculate_psnr(denoised, clean)
            ssim_val = calculate_ssim(denoised, clean)
            psnr_list.append(psnr_val)
            ssim_list.append(ssim_val)

mean_psnr = np.mean(psnr_list)
mean_ssim = np.mean(ssim_list)

print(f"Test PSNR: {mean_psnr:.2f} dB")
print(f"Test SSIM: {mean_ssim:.4f}")

Test PSNR: 20.25 dB
Test SSIM: 0.6026
